# Imports

In [1]:
import os
import time
import random
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

import gym
from gym import spaces
from PIL import Image
from collections import deque, namedtuple
from IPython.display import HTML
from IPython import display
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
from environments import Snake
from visual_utils import AgentViz
from methods import DistQNetwork, ReplayMemory
from agents import SnakeDistDQNAgent

# NN training

In [3]:
aa = SnakeDistDQNAgent(model_name="distdqn_baseline", state_shape=[8, 8, 3], num_atoms=51)

In [4]:
aa.set_parameters(max_episode_length=2000, replay_memory_size=100000, replay_start_size=10000,
                  discount_factor=0.99, final_eps=0.01, annealing_steps=500000)

In [5]:
aa.train(gpu_id=1, from_epoch=0)

frame count: 4560
average reward: -0.908
epsilon: 0.991
average lifetime: 9.12
-------------------------------
frame count: 9071
average reward: -0.884
epsilon: 0.982
average lifetime: 9.022
-------------------------------
frame count: 13220
average reward: -0.894
epsilon: 0.974
average lifetime: 8.298
-------------------------------
frame count: 17942
average reward: -0.888
epsilon: 0.964
average lifetime: 9.444
-------------------------------
frame count: 22759
average reward: -0.9
epsilon: 0.955
average lifetime: 9.634
-------------------------------
frame count: 27843
average reward: -0.88
epsilon: 0.945
average lifetime: 10.168
-------------------------------
frame count: 32767
average reward: -0.878
epsilon: 0.935
average lifetime: 9.848
-------------------------------
frame count: 37698
average reward: -0.876
epsilon: 0.925
average lifetime: 9.862
-------------------------------
frame count: 42250
average reward: -0.858
epsilon: 0.916
average lifetime: 9.104
--------------------

frame count: 1316911
average reward: 14.274
epsilon: 0.01
average lifetime: 183.852
-------------------------------
frame count: 1408907
average reward: 14.364
epsilon: 0.01
average lifetime: 183.992
-------------------------------
frame count: 1501596
average reward: 14.796
epsilon: 0.01
average lifetime: 185.378
-------------------------------
frame count: 1607151
average reward: 16.034
epsilon: 0.01
average lifetime: 211.11
-------------------------------
frame count: 1708101
average reward: 15.812
epsilon: 0.01
average lifetime: 201.9
-------------------------------
frame count: 1807289
average reward: 15.762
epsilon: 0.01
average lifetime: 198.376
-------------------------------
frame count: 1905728
average reward: 15.99
epsilon: 0.01
average lifetime: 196.878
-------------------------------
frame count: 2017596
average reward: 17.492
epsilon: 0.01
average lifetime: 223.736
-------------------------------
frame count: 2128649
average reward: 17.412
epsilon: 0.01
average lifetime: 

KeyboardInterrupt: 

# Visualization

In [1]:
aa.play(gpu_id=1, max_episode_length=100, from_epoch=234)

NameError: name 'aa' is not defined